Videos 1-5

https://en.wikipedia.org/wiki/Regression_analysis

In [81]:
import pandas as pd
import math
import numpy as np
from sklearn import preprocessing, cross_validation , svm
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression

In [100]:
df = pd.read_csv("NSE_data.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-21,10919.349609,10987.450195,10885.750000,10961.849609,10961.849609,280100
1,2019-01-22,10949.799805,10949.799805,10864.150391,10922.750000,10922.750000,291800
2,2019-01-23,10931.049805,10944.799805,10811.950195,10831.500000,10831.500000,0
3,2019-01-24,10844.049805,10866.599609,10798.650391,10849.799805,10849.799805,350300
4,2019-01-25,10859.750000,10931.700195,10756.450195,10780.549805,10780.549805,449500


In [101]:
df.describe()

,Date,Open,High,Low,Close,Adj Close,Volume
count,242,242,242,242,242,242,242
unique,242,240,241,241,239,239,142
top,2019-04-24,null,null,null,11844.099609,11844.099609,600
freq,1,2,2,2,2,2,29


Regression does not consider the relationship between the features, so we need to make our own new features which will show the relationship between different features.

In [102]:
df = df.fillna(axis = 1, method = "ffill")

In [103]:
def cus_fill(data):
    list_data = list(data)
    for i in range(0,len(list_data)):
        if list_data[i] == "null":
            list_data[i] = list_data[i-1]
    return list_data

for col in df.columns:
    df[col] = cus_fill(df[col])

In [104]:
df['HighLow_PRCT'] = (df['High'].astype(float)-df['Close'].astype(float))/df['Close'].astype(float)*100
df['PCT_change'] = (df['Close'].astype(float) - df['Open'].astype(float))/df['Open'].astype(float)*100

In [105]:
df_new = df[['Adj Close','HighLow_PRCT','PCT_change','Open']]
df_new.head()

,Adj Close,HighLow_PRCT,PCT_change,Open
0,10961.849609,0.233543,0.389217,10919.349609
1,10922.750000,0.247646,-0.247035,10949.799805
2,10831.500000,1.046021,-0.910707,10931.049805
3,10849.799805,0.154840,0.053024,10844.049805
4,10780.549805,1.402066,-0.729300,10859.750000


Briefly, feature is input; label is output. This applies to both classification and regression problems.
A feature is one column of the data in your input set. For instance, if you're trying to predict the type of pet someone will choose, your input features might include age, home region, family income, etc. The label is the final choice, such as dog, fish, iguana, rock, etc.

Once you've trained your model, you will give it sets of new input containing those features; it will return the predicted "label" (pet type) for that person.

In [106]:
forcast_col = 'Adj Close'

In [107]:
# using this we are saying that we want to predict for the next 10 % days.

forecast_out = int(math.ceil(0.01*len(df_new)))

In [108]:
df_new['label'] = df_new[forcast_col].shift(-forecast_out)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [119]:
df_new.head(2)

,Adj Close,HighLow_PRCT,PCT_change,Open,label
0,10961.849609,0.233543,0.389217,10919.349609,10849.799805
1,10922.750000,0.247646,-0.247035,10949.799805,10780.549805


In [129]:
df_new.dropna(inplace = True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


np array,ready features and label, 

In [130]:
features = np.array(df_new.drop(['label'], axis = 1))
#features

In [131]:
label = np.array(df_new.label)
#label

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html

preprocessing is used for converting the data into normal(Gaussian) Distribution. 

In [132]:
df_new.dropna(inplace = True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [133]:
scaled_X = preprocessing.scale(features)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [134]:
#features = features[:-forecast_out+1]

In [135]:
y = np.array(df_new['label'])

In [136]:
print(len(features),len(y))

239 239


In [137]:
# http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.cross_validation.train_test_split.html

feature_train, feature_test, y_train, y_test = cross_validation.train_test_split(features, y,test_size=0.20, random_state=42)

In [138]:
regre = LinearRegression().fit(feature_train, y_train)

In [140]:
regre.score(feature_test, y_test)

0.88381606035613158

In [141]:
regre2 = svm.SVR().fit(feature_train, y_train)

In [142]:
regre2.score(feature_test, y_test)

-0.038806382904803538